# 지속가능경영보고서 크롤링 (110/200개 기업) https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp

#### 1. 지속가능경영보고서접속링크를 포함하는 csv 파일 저장
-> 회사명, 발행년도, 코드, 페이지접속링크

In [ ]:
import pandas as pd
import requests
import json

# 크롤링할 년도 범위 및 페이지 범위 설정
years = range(2013, 2024)
pages = range(1, 19)  # 예시로 1부터 18페이지까지 크롤링

# 결과를 저장할 리스트 초기화
data_df = []

# 년도와 페이지를 모두 반복하며 크롤링 수행
for year in years:
    for page in pages:
        url = 'https://esg.krx.co.kr/contents/99/ESG99000001.jspx'
        payload = {
            'sch_yy': year,
            'sch_tp': 'N', 
            'pagePath': '/contents/02/02030000/ESG02030000.jsp', 
            'code': '02/02030000/esg02030000_01', 
            'curPage': page
        }

        response = requests.post(url, data=payload, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'})

        # 응답 확인
        if response.status_code == 200:
            python_data = json.loads(response.text)
            for ele in python_data['result']:
                data_df.append([ele['com_abbrv'], ele['yy'], ele['save_file_nm']])
        else:
            print(f"페이지 {page}, 년도 {year} 크롤링 실패")

# 데이터프레임 생성
columns = ['회사명', '발행년도', '코드']
df = pd.DataFrame(data_df, columns=columns)
df['페이지접속링크'] = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno='+df['코드']+'&docno=&viewerhost=&viewerport='

# 코스피 200 기업명 리스트 가져오기
f = open('top200_name_list.csv','r',encoding='utf-8')
data = f.read().split('\n')

# 데이터프레임의 조직명과 코스피 200 리스트의 회사명을 비교하여 코스피 200에 속하는 행만 추출
kospi200_companies_df = df[df['회사명'].isin(data)]

# 저장
kospi200_companies_df.to_csv('지속가능경영보고서접속링크.csv',index=False)

#### 2. selenium 사용해 지속가능경영보고서 pdf 다운로드 링크 추가
기존 csv 파일에 접근하여 pdf 다운로드할 수 있는 링크를 추가하여 csv 파일로 다시 저장  

-> 회사명, 발행년도, 코드, 페이지접속링크 pdf다운로드링크  

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

df = pd.read_csv('지속가능경영보고서접속링크.csv')

# 웹 드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 새로운 열 생성 및 초기화
df['pdf다운로드링크'] = ''

# 데이터프레임 반복문
for index, row in df.iterrows():
    # 웹사이트에 접속
    driver.get(row['페이지접속링크'])

    # 요소 클릭
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'attachedDoc'))).click()
    except:
        print("요소를 찾을 수 없습니다.")
        print(index, row)
        print(row['페이지접속링크'])
        continue

    # 두 번째 옵션 클릭
    try:
        options = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.XPATH, "//select[@id='attachedDoc']/option")))
        if len(options) > 1:
            options[1].click()  # 두 번째 옵션 선택
    except:
        print("옵션을 찾을 수 없습니다.")
        continue

    # iframe 전환
    try:
        driver.switch_to.frame('docViewFrm')
    except:
        print("iframe을 찾을 수 없습니다.")
        continue

    # PDF 다운로드 링크 가져오기
    try:
        pdf_download_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div/div/a"))).get_attribute('href')
        # 데이터프레임에 저장
        df.at[index, 'pdf다운로드링크'] = pdf_download_link
    except:
        print("PDF 다운로드 링크를 찾을 수 없습니다.")
        df.at[index, 'pdf다운로드링크'] = None

# 웹 드라이버 종료
driver.close()
driver.quit()

# df.isnull().sum() # 결측치 개수 확인

# 결측치인 행의 인덱스 반환 -> 24
df['pdf다운로드링크'][df['pdf다운로드링크'].isnull()]

# 결측치가 존재하는 행  제거
df = df.drop(24)

df.to_csv('지속가능경영보고서다운로드링크추가.csv',index=False)

#### 3. pdf 파일 저장
해당 경로에 pdf 파일 저장할 폴더"pdf" 생성 후, pdf 저장

In [ ]:
# pdf 다운로드
import os
import urllib.request
import traceback
import pandas as pd

df2 = pd.read_csv('지속가능경영보고서다운로드링크추가.csv')

total_info = [tuple(row) for row in df2.values] # 데이터프레임을 리스트 안의 튜플 데이터 형태로 변환

dir = './pdf/' # pdf 파일 저장할 경로 설정
if not os.path.exists(dir): # 폴더 없는 경우에 생성하는 코드
    os.makedirs(dir)

for i in range(len(total_info)):
    if i==0:
        print('pdf 다운로드 시작')
    try:
        file_name = total_info[i][0]+'_'+str(total_info[i][1])+'.pdf'
        urllib.request.urlretrieve(total_info[i][4], dir+file_name)
    except:
        print(f"error : {total_info[i][0]}")
        print(traceback.format_exc())
        i += 1 # 에러가 발생한 경우, 해당 항목부터 다시 크롤링할 수 있도록 i를 1 증가시킴
    if i!=0 and i%100==0:
        print(f'pdf 다운로드 진행률 : {i}/{len(total_info)}') # 100개 다운로드마다 알림
print('완료')

#### 4. pdf 파일에 접근해 텍스트만 가져와 전처리하여 기존 csv 파일에 저장
-> 회사명, 발행년도, 코드, 페이지접속링크 pdf다운로드링크 내용  

In [ ]:
import os
import fitz
import kss
import pandas as pd

df3 = pd.read_csv('지속가능경영보고서다운로드링크추가.csv')

pdf_path = os.getcwd() + '\\pdf\\'

final_list = []
for idx, row in df3.iterrows():
    pdf_file_path = pdf_path + row['회사명'] + '_' + str(row['발행년도']) + '.pdf'
    try:
        with fitz.open(pdf_file_path) as doc:
            contents = ''
            for page in doc:
                contents += page.get_text()
            sents = kss.split_sentences(contents)
            sents = [sent.replace('\n', '') for sent in sents]

            new_sentences = []
            for sent in sents:
                # Split the sentence by both '.' and 'ㆍ'
                sub_sentences = sent.replace('ㆍ', '.').split('.')
                # Add each sub-sentence to the new list
                new_sentences.extend(sub_sentences)
            new_sentences = [sent.strip() for sent in new_sentences if sent.strip()]

            sentences = [sent for sent in new_sentences if sent.endswith('다')] # 다로 끝나는 것들만 모아서 문장취급함
            final_list.append(sentences)
    except:
        print('해당파일 제외 :', pdf_file_path)
        # 파일을 열 수 없는 경우 None 값을 추가합니다.
        final_list.append(None)

# 데이터프레임을 복제하여 새로운 데이터프레임을 생성합니다.
new_df = df3.copy()

# '내용' 열을 final_list의 각 항목으로 설정합니다.
new_df['내용'] = final_list

# 수정된 데이터프레임을 출력합니다.
new_df